In [0]:
# 1. Завантаження даних

delta_input_path = "/Volumes/workspace/default/olist_delta_2"
olist_df = spark.read.format("delta").load(delta_input_path)

print("Rows:", olist_df.count())
display(olist_df.limit(5))


In [0]:
# 2. Вибір числових ознак

from pyspark.sql.functions import udf, col
from pyspark.sql.types import ArrayType, DoubleType

def vector_to_list_safe(v):
    if v is None:
        return None
    try:
        return [float(x) for x in v.toArray()]
    except Exception:
        return None

vector_to_list_udf = udf(vector_to_list_safe, ArrayType(DoubleType()))

if "numeric_scaled" in olist_df.columns:
    olist_df = olist_df.withColumn("numeric_array", vector_to_list_udf(col("numeric_scaled")))
    num_features = 6
    numeric_cols = [f"num_{i+1}" for i in range(num_features)]
    for i, cname in enumerate(numeric_cols):
        olist_df = olist_df.withColumn(cname, col("numeric_array")[i])
else:
    numeric_cols = [c for c, t in olist_df.dtypes if t in ("double", "float", "int", "bigint")][:30]

print("Numeric cols:", numeric_cols[:30])


In [0]:
import pandas as pd

# Беремо підвибірку ~10% для кореляції, щоб не грузити пам'ять
sample_pd = olist_df.select(numeric_cols).sample(False, 0.1, seed=42).toPandas()
corr = sample_pd.corr()
display(corr)

# Знайдемо сильно корельовані пари |r| > 0.8
thr = 0.8
high_corr = [
    (c1, c2, corr.loc[c1, c2])
    for i, c1 in enumerate(corr.columns)
    for j, c2 in enumerate(corr.columns)
    if j > i and abs(corr.loc[c1, c2]) > thr
]
print("High correlations:", high_corr)

# ВІДБІР ОЗНАК: видаляємо другу ознаку з пари (фільтраційний метод 1 — correlation)
for c1, c2, r in high_corr:
    print(f"Drop {c2} because of high correlation with {c1} (r={r:.2f})")
    if c2 in olist_df.columns:
        olist_df = olist_df.drop(c2)
    if c2 in numeric_cols:
        numeric_cols.remove(c2)

print("Numeric cols after correlation-based selection:", len(numeric_cols))


In [0]:
# 4. Інженерія нових ознак (мінімум 5)

from pyspark.sql.functions import when, lit, mean as _mean
import pyspark.sql.functions as F

# 1) Ознаки, пов’язані з часом доставки
if "delivery_time_days_calc" in olist_df.columns:
    avg_delivery = olist_df.select(_mean(col("delivery_time_days_calc"))).first()[0] or 0.0
    olist_df = olist_df.withColumn(
        "delivery_diff_avg",
        col("delivery_time_days_calc") - lit(avg_delivery)
    )
    olist_df = olist_df.withColumn(
        "fast_delivery",
        when(col("delivery_time_days_calc") < 3, 1).otherwise(0)
    )
else:
    olist_df = olist_df.withColumn("delivery_diff_avg", lit(0.0)) \
                       .withColumn("fast_delivery", lit(0))

# 2) Квадрат першої числової ознаки
olist_df = olist_df.withColumn("num_1_sq", (col("num_1") ** 2).cast("double"))

# 3) Взаємодія між двома фічами
olist_df = olist_df.withColumn(
    "num2_num3_interaction",
    (col("num_2") * col("num_3")).cast("double")
)

# 4) Середнє двох перших числових ознак
olist_df = olist_df.withColumn(
    "num_mean_12",
    ((col("num_1") + col("num_2")) / 2).cast("double")
)

# 5) Логарифм від num_1 (робимо log1p(abs(x)))
olist_df = olist_df.withColumn(
    "log_num_1",
    F.log1p(F.abs(col("num_1")))
)

# Додаємо нові фічі до списку числових
for f in ["delivery_diff_avg", "fast_delivery",
          "num_1_sq", "num2_num3_interaction",
          "num_mean_12", "log_num_1"]:
    if f in olist_df.columns and f not in numeric_cols:
        numeric_cols.append(f)

print("Нові ознаки додано. Всього числових:", len(numeric_cols))


In [0]:
# 5. Масштабування + Відбір ознак (Метод 2 — Variance Threshold)

import pyspark.sql.functions as F

# Обчислюємо дисперсію для кожної фічі
var_row = olist_df.select([F.variance(col(c)).alias(c) for c in numeric_cols]).collect()[0].asDict()
var_items = [(c, float(var_row.get(c) or 0.0)) for c in numeric_cols]

# Фільтраційний метод 2: відбір за порогом дисперсії
variance_threshold = 0.01
selected_by_variance = [c for c, v in var_items if v > variance_threshold]

print("Ознаки, відібрані за порогом дисперсії (> 0.01):")
print(selected_by_variance)

# Для масштабування фічs з найвищою дисперсією
var_items_sorted = sorted(var_items, key=lambda x: x[1], reverse=True)
topk = [c for c, _ in var_items_sorted[:50]]

# mean і std тільки для top-k
stats = olist_df.select(
    *[F.mean(col(c)).alias(f"{c}_mean") for c in topk],
    *[F.stddev(col(c)).alias(f"{c}_std") for c in topk]
).collect()[0].asDict()

# Масштабування (z-score)
for c in topk:
    mean_val = stats.get(f"{c}_mean") or 0.0
    std_val = stats.get(f"{c}_std") or 1.0
    if std_val == 0:
        std_val = 1.0
    olist_df = olist_df.withColumn(f"{c}_scaled", (col(c) - mean_val) / std_val)

scaled_cols = [f"{c}_scaled" for c in topk]

from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(
    inputCols=scaled_cols,
    outputCol="numeric_scaled_features",
    handleInvalid="skip"
)
olist_df = assembler.transform(olist_df)

print(f"Масштабування виконано для {len(scaled_cols)} фіч.")
display(olist_df.select("numeric_scaled_features").limit(3))


In [0]:

import numpy as np
from sklearn.decomposition import PCA

# вибірка для аналізу
pca_sample_pd = olist_df.select(scaled_cols).sample(False, 0.02, seed=42).toPandas()

pca_input_cols = scaled_cols[:20]
X_pca = pca_sample_pd[pca_input_cols].values

pca = PCA(n_components=3, random_state=42)
X_pca_trans = pca.fit_transform(X_pca)

print("PCA (sklearn) виконано успішно.")
print("Пояснена дисперсія по компонентах:", pca.explained_variance_ratio_)
print("Сумарна пояснена дисперсія:", pca.explained_variance_ratio_.sum())

# Feature selection через PCA loadings
loadings = np.abs(pca.components_).sum(axis=0)  # сумарний внесок кожної фічі
pca_feature_importance = sorted(
    zip(pca_input_cols, loadings),
    key=lambda x: x[1],
    reverse=True
)[:10]

print("Топ-10 найважливіших ознак за PCA loadings:")
for name, score in pca_feature_importance:
    print(f"{name}: {score:.6f}")

pca_sample_pd[["pca_1", "pca_2", "pca_3"]] = X_pca_trans
display(pca_sample_pd[["pca_1", "pca_2", "pca_3"]].head())


In [0]:
%pip install umap-learn

In [0]:
# 7. Додаткове зменшення розмірності: UMAP 

import umap

umap_model = umap.UMAP(n_components=2, random_state=42)
X_umap = umap_model.fit_transform(X_pca)

print("UMAP виконано успішно. Розмірність:", X_umap.shape)

pca_sample_pd["umap_1"] = X_umap[:, 0]
pca_sample_pd["umap_2"] = X_umap[:, 1]

display(pca_sample_pd[["umap_1", "umap_2"]].head())



In [0]:
# 8. ВІЗУАЛІЗАЦІЯ UMAP
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


print("Візуалізація UMAP...")

# Налаштування стилю
sns.set_style("whitegrid")

# Створення фігури та осей
plt.figure(figsize=(10, 8))

# Побудова точкової діаграми
sns.scatterplot(
    x='umap_1', 
    y='umap_2', 
    data=pca_sample_pd, 
    s=10, # Розмір точок
    alpha=0.6, # Прозорість
    
)

plt.title('UMAP Проєкція даних (2D)', fontsize=16)
plt.xlabel('UMAP Dimension 1')
plt.ylabel('UMAP Dimension 2')
plt.grid(True)
plt.show()

In [0]:
# 8. Збереження результату

output_path = "/Volumes/workspace/default/olist_delta_3"

olist_df.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(output_path)

print("Результат збережено у:", output_path)
